<a href="https://colab.research.google.com/github/MatP-DS/MasterThesis/blob/main/export_all_s2_band_and_indices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import os
from datetime import datetime
import os
from google.colab import drive


In [ ]:
# ------------------------------
# STEP 1: Setup
# ------------------------------
ee.Authenticate()
ee.Initialize(project='ee-masterthesisdeforestation')

# Safely mount Google Drive only if not mounted
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# AOI
aoi = ee.Geometry.Rectangle([-56.2, -11.8, -56.0, -11.6])

# ------------------------------
# STEP 2: Define cloud masking
# ------------------------------
def maskS2sr(img):
    scl = img.select('SCL')
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9))
    return img.updateMask(mask).copyProperties(img, ["system:time_start"])

# ------------------------------
# STEP 3: Compute all indices
# ------------------------------
def add_all_indices(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': img.select('B8'),
            'RED': img.select('B4'),
            'BLUE': img.select('B2')
        }).rename('EVI')
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    nbr = img.normalizedDifference(['B8', 'B12']).rename('NBR')
    savi = img.expression(
        '((NIR - RED) / (NIR + RED + 0.5)) * 1.5', {
            'NIR': img.select('B8'),
            'RED': img.select('B4')
        }).rename('SAVI')
    return img.addBands([ndvi, evi, ndwi, nbr, savi])


In [ ]:
# ------------------------------
# STEP 4: Monthly export loop
# ------------------------------
output_folder = '01_sentinel2/full_feature_stack'
local_check_dir = '/content/drive/MyDrive/MasterThesis/01_raw_data/' + output_folder
os.makedirs(local_check_dir, exist_ok=True)

years = [2020, 2021, 2022, 2023]

for year in years:
    for month in range(1, 13):
        start = ee.Date.fromYMD(year, month, 1)
        end = start.advance(1, 'month')
        filename = f'S2_full_{year}_{month:02d}.tif'
        local_path = os.path.join(local_check_dir, filename)

        # Skip if already exists
        if os.path.exists(local_path):
            print(f"⏭️ Skipping existing: {filename}")
            continue

        # Load collection
        collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                      .filterDate(start, end)
                      .filterBounds(aoi)
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
                      .map(maskS2sr)
                      .map(add_all_indices)
                      .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12',
                               'NDVI', 'EVI', 'NDWI', 'NBR', 'SAVI']))

        composite = collection.mean().clip(aoi)

        # Skip if empty
        if composite.bandNames().size().getInfo() == 0:
            print(f"No data in {year}-{month:02d}")
            continue

        # Export task
        task = ee.batch.Export.image.toDrive(
            image=composite.toFloat(),
            description=f'Export_S2_full_{year}_{month:02d}',
            folder=output_folder.split("/")[-1],
            fileNamePrefix=f'S2_full_{year}_{month:02d}',
            region=aoi,
            scale=30,
            fileFormat='GeoTIFF',
            maxPixels=1e13
        )
        task.start()
        print(f"Export started: {filename}")

No data in 2020-01
Export started: S2_full_2020_02.tif
Export started: S2_full_2020_03.tif
Export started: S2_full_2020_04.tif
Export started: S2_full_2020_05.tif
Export started: S2_full_2020_06.tif
Export started: S2_full_2020_07.tif
Export started: S2_full_2020_08.tif
Export started: S2_full_2020_09.tif
Export started: S2_full_2020_10.tif
Export started: S2_full_2020_11.tif
Export started: S2_full_2020_12.tif
Export started: S2_full_2021_01.tif
No data in 2021-02
Export started: S2_full_2021_03.tif
Export started: S2_full_2021_04.tif
Export started: S2_full_2021_05.tif
Export started: S2_full_2021_06.tif
Export started: S2_full_2021_07.tif
Export started: S2_full_2021_08.tif
Export started: S2_full_2021_09.tif
Export started: S2_full_2021_10.tif
Export started: S2_full_2021_11.tif
Export started: S2_full_2021_12.tif
Export started: S2_full_2022_01.tif
Export started: S2_full_2022_02.tif
Export started: S2_full_2022_03.tif
Export started: S2_full_2022_04.tif
Export started: S2_full_20